In [ ]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
sys.path.append('d:/ariel2/code/core/')
sys.path.append('/kaggle/input/my-ariel2-library')
import kaggle_support as kgs
import ariel_model
import ariel_simple
import ariel_gp
import ariel_load
import ariel_load_FGS
import os
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib
from tqdm import tqdm
import seaborn as sns
import pandas as pd
kgs.debugging_mode = 1
kgs.profiling=False
kgs.n_workers = 6

import warnings
warnings.filterwarnings(
    "ignore",
    message=r".*use_inf_as_na option is deprecated.*",
    category=FutureWarning,
)

In [ ]:
# Parameters in GP
# 0: 0 t0
# 1: 1 t0
# 2: alpha
# 3: beta
# 4: i
# 5: 0 u0 
# 6: 1 u0
# 7: 0 u1
# 8: 1 u1
# 9: 1 us0
# 10:1 us1

# Parameters in simple
# 0: 0 t0
# 1: alpha
# 2: beta
# 3: i
# 4: 0 rp
# 5: 0 u0
# 6: 0 u1
# 7: 1 rp
# 8: 1 u0
# 9: 1 u1
# 9-xx: poly
# -1: 1 t0

In [ ]:
def remove_outlier(mat, threshold):
    while True:
        mu = np.mean(mat,0)
        std = np.std(mat,0)
        scaled = np.abs(mat-mu)/std
        #plt.figure()
        #plt.plot(scaled)
        if np.max(scaled)>threshold:
            max_per_row = np.max(scaled,1)
            to_remove = np.argmax(max_per_row)
            mat = np.delete(mat,to_remove,0)
        else:
            break
    return mat

In [ ]:
train_data = kgs.load_all_train_data()
train_data2 = train_data
train_data = train_data[:100]

In [ ]:
importlib.reload(ariel_gp)
model = ariel_model.baseline_model()
model = model.model
model.model_options.output_model = True
model.model_options.use_training_labels = True
model.model_options.n_samples_sigma_est = 2
model.starter_model.model.expose_Rp_fudge = True
model.starter_model.model.do_regularization = False
def modify_func(model, cc):
        model.m['signal'].m['main'].m['transit'].cov_override = cc[0]
        model.m['signal'].m['main'].m['transit'].mu_override = cc[1]
        return model
model.model_options.modify_func = modify_func
model.model_options.modify_func_input = (None,None)
model.train(train_data)

In [ ]:
modelr = copy.deepcopy(model)
modelr.model_options.n_iter = 0
modelr.run_in_parallel = False
modelr.infer(train_data[0:1]);

In [ ]:
data = copy.deepcopy(train_data)

In [ ]:
kgs.sanity_checks_active = False
for ii in range(50):
    if ii==5:
        data = copy.deepcopy(train_data2)
    data = model.infer(data)
    transit_parameters = []
    for d in data:
        d.diagnostics['starting_par'] = d.diagnostics['par_mean']
        modelr.results['model_samples'].m['signal'].set_parameters(d.diagnostics['par_samples'])
        transit_parameters.append(modelr.results['model_samples'].m['signal'].m['main'].m['transit'].get_parameters()[:,0])
    transit_parameters = np.array(transit_parameters)
    
    transit_parameters = remove_outlier(transit_parameters,4)
    print(transit_parameters.shape)

    plt.figure()
    plt.imshow(np.corrcoef(transit_parameters.T))
    plt.colorbar()
    sns.pairplot(pd.DataFrame(transit_parameters), diag_kind="hist", corner=True, height=1.5)
    plt.pause(0.0001)

    cov_est = np.cov(transit_parameters.T)
    print(np.sqrt(np.diag(cov_est)))
    mu_est = np.mean(transit_parameters,0)
    def modify_func(model, cc):
        model.m['signal'].m['main'].m['transit'].cov_override = cc[0]
        model.m['signal'].m['main'].m['transit'].mu_override = cc[1]
        # cur_params = model.m['signal'].m['main'].m['transit'].get_parameters()
        # cur_params[-model.m['signal'].m['main'].m['transit'].number_of_extra_parameters:,0] = cc[1]
        # model.m['signal'].m['main'].m['transit'].set_parameters(cur_params)
        return model
    model.model_options.modify_func = modify_func
    model.model_options.modify_func_input = [cov_est, mu_est]
    
    kgs.dill_save(kgs.temp_dir + f'transit_model_tuning_Rp_fudge_{ii}.pickle', (cov_est, mu_est))

In [ ]:
# import subprocess

# # Hibernate Windows
# # subprocess.run(
# #     ["/mnt/c/Windows/System32/shutdown.exe", "/a"],  # add "/f" to force-close apps
# #     check=True
# # )
# subprocess.run(
#     ["/mnt/c/Windows/System32/shutdown.exe", "/h"],  # add "/f" to force-close apps
#     check=True
# )